# World Cup 2018 Statistics

This jupyter notebook takes historic match data of all world cups and generates some statistics that may help you with all these betting games at work you are confronted soon.

Let's import the CSV file and take a look at the data.

In [1]:
import qgrid
import pandas as pd

df = pd.read_csv('historic_data.csv')

df.head()

,team,home_team,home_goals,away_goals,away_team,game_type,year
0,Brazil,Brazil,0,3,Netherlands,Game Over 3rd Place,2014
1,Brazil,Brazil,1,7,Germany,Semifinal,2014
2,Brazil,Brazil,2,1,Colombia,quarterfinal,2014
3,Brazil,Brazil,1,1,Chile,Round Of 16,2014
4,Brazil,Brazil,4,1,Cameroon,Group Phase,2014


To make analyzation easier, let's apply some extra data.

In [2]:
def row_to_pov_result(row):
    team_goals = row['home_goals'] if row['home_team'] == row['team'] else row['away_goals']
    opponent_goals = row['away_goals'] if row['home_team'] == row['team'] else row['home_goals']        
    return '{}-{}'.format(team_goals, opponent_goals)


def row_to_winner_result(row):
    home_goals = row['home_goals']
    away_goals = row['away_goals']
    
    winner_goals = home_goals if home_goals > away_goals else away_goals
    loser_goals = away_goals if home_goals > away_goals else home_goals
    return '{}-{}'.format(winner_goals, loser_goals)


def row_to_is_team_winner(row):
    if row['home_goals'] > row['away_goals']:
        return row['home_team'] == row['team']
    else:
        return row['away_team'] == row['team']

    
def row_to_game_status(row):
    if row['home_goals'] == row['away_goals']:
        return 'DRAW'
    elif row['home_goals'] > row['away_goals']:
        return 'WIN' if row['home_team'] == row['team'] else 'LOSE'
    else:
        return 'WIN' if row['away_team'] == row['team'] else 'LOSE'
    

df['game_status'] = df.apply(row_to_game_status, axis=1)
df['pov_result'] = df.apply(row_to_pov_result, axis=1)
df['winner_result'] = df.apply(row_to_winner_result, axis=1)

In [3]:
df.head()

,team,home_team,home_goals,away_goals,away_team,game_type,year,game_status,pov_result,winner_result
0,Brazil,Brazil,0,3,Netherlands,Game Over 3rd Place,2014,LOSE,0-3,3-0
1,Brazil,Brazil,1,7,Germany,Semifinal,2014,LOSE,1-7,7-1
2,Brazil,Brazil,2,1,Colombia,quarterfinal,2014,WIN,2-1,2-1
3,Brazil,Brazil,1,1,Chile,Round Of 16,2014,DRAW,1-1,1-1
4,Brazil,Brazil,4,1,Cameroon,Group Phase,2014,WIN,4-1,4-1


## Analysis

They always say that the easy bet is 2-1 on the favorite team. But the better bet seems to be 1-0...

In [4]:
df\
    .groupby(['winner_result'])['winner_result']\
    .count()\
    .reset_index(name='count')\
    .sort_values(['count'], ascending=False)\
    .head(10)

,winner_result,count
1,1-0,192
4,2-1,140
3,2-0,96
0,0-0,89
7,3-1,89
2,1-1,78
6,3-0,53
8,3-2,48
5,2-2,35
11,4-1,34


... which is also right if we only look at the football games after the 90s. So change your bets asap.

In [5]:
df\
    .query('year > 1990')\
    .groupby(['winner_result'])['winner_result']\
    .count()\
    .reset_index(name='count')\
    .sort_values(['count'], ascending=False)\
    .head(10)

,winner_result,count
1,1-0,102
4,2-1,70
3,2-0,52
2,1-1,40
0,0-0,39
7,3-1,34
6,3-0,26
5,2-2,24
8,3-2,21
10,4-0,16


## The Tool

Search for the most frequent results of a team depending on phase of the tournament they are in

In [6]:
data = df\
    .groupby(['team', 'game_type', 'pov_result', 'game_status'])['team']\
    .count()\
    .reset_index(name='count')\
    .sort_values(['count'], ascending=False)\

data.head(15)

,team,game_type,pov_result,game_status,count
177,England,Group Phase,0-0,DRAW,10
82,Brazil,Group Phase,1-0,WIN,8
183,England,Group Phase,2-0,WIN,7
535,Uruguay,Group Phase,0-0,DRAW,7
87,Brazil,Group Phase,2-1,WIN,7
8,Argentina,Group Phase,1-0,WIN,7
180,England,Group Phase,1-0,WIN,7
467,Spain,Group Phase,3-1,WIN,7
250,Germany,Group Phase,1-0,WIN,6
53,Belgium,Group Phase,1-0,WIN,6


Let's put this data in a filterable tool.

In [7]:
qgrid.show_grid(data, show_toolbar=False)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…